In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PPlib import jpg_to_face, filter_faces, jpg_to_lmdb

In [ ]:
# define constants
input_dir_sml = 'data/train_small/'
face_data = "valid_small.csv"
output_dir_sml = 'data/train_small_clean/'
db_path = 'data/train_small_lmdb/train_small.lmdb'

i_start = 0
num_img = 6955
q_mn = 0.98
ar_mx = 1
ar_mn = 0.5
px_mn = 4096
npx = 224

In [ ]:
# find all images with exactly one face
valid_id, valid_bbox_sml, valid_q_sml, valid_ar_sml, valid_px_sml, bad_id = jpg_to_face(input_dir_sml, i_start, num_img)

In [ ]:
# convert to arrays
num_valid = len(valid_id)

valid_q = np.array(valid_q_sml).reshape(num_valid,1)
valid_ar = np.array(valid_ar_sml).reshape(num_valid,1)
valid_px = np.array(valid_px_sml).reshape(num_valid,1)
valid_id = np.array(valid_id).reshape(num_valid,1)
valid_bbox = np.array(valid_bbox_sml).reshape(num_valid,4)

# save data
valid_small = np.hstack((valid_id, valid_q, valid_ar, valid_px, valid_bbox))
np.savetxt(face_data, valid_small, delimiter=",")

# apply filters
q_filt = valid_q > q_mn
ar_filt_mx = valid_ar < ar_mx
ar_filt_mn = valid_ar > ar_mn
px_filt = valid_px > px_mn
all_filt = (q_filt * ar_filt_mx * ar_filt_mn * px_filt)

# determine baseline
print(num_valid)

# determine filtered
print(sum(all_filt))

# determine additional filtering
plt.figure(1)
plt.hist(valid_q_sml)
print(sum(q_filt))
# conclusion: q > 0.97

plt.figure(2)
plt.hist(valid_ar_sml)
print(sum(ar_filt_mx))
# conclusion: ar < 1

plt.figure(3)
plt.hist(valid_ar_sml)
print(sum(ar_filt_mn))
# conclusion: ar > 0.5

plt.figure(4)
plt.hist(valid_px_sml)
print(sum(px_filt))
# conclusion: px > 4096

In [ ]:
# get data from spreadsheet
valid_small = np.loadtxt(face_data, delimiter =',')

valid_id = valid_small[:,0]
valid_q = valid_small[:,1]
valid_ar = valid_small[:,2]
valid_px = valid_small[:,3]
valid_bbox = valid_small[:,4:]

num_valid = len(valid_id)

# convert images to standard form
flag = filter_faces(input_dir_sml, output_dir_sml, valid_id, valid_bbox, valid_q, valid_ar, valid_px, num_valid, ar_mx, ar_mn, q_mn, px_mn, npx)

In [ ]:
# convert images into database
jpg_to_lmdb(input_dir_sml, db_path)